In [1]:
import requests
from bs4 import BeautifulSoup
from utils import *
import pandas as pd
import time
from time import sleep
import random
import io 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
import json
import os.path

In [ ]:
# generate HTML files and dataframe 
from collector import *

In [ ]:
# generate TSV files from HTML
from parser import *

In [119]:
# generate all index files and vocabulary
from index import *

KeyboardInterrupt: 

In [2]:
from main import * 

In [3]:
def search_1(query, top = 10):
    return top_k(search_no_score(query, vocabulary, stop_words, inverted_index), df_movies, top, no_score=True)

In [4]:
search_1('world war')

NameError: name 'df_movies' is not defined

# Inverted index 2

In [123]:
def search_2(query, top = 10):
    return f.top_k(f.search_with_tfidf(query, vocabulary, inverted_index_2, doc_norm), df_movies, top)

In [124]:
search_2('Kill Bill', 20)

,Score,Title,Intro,URL
0,0.10,The White Buffalo,"The White Buffalo is a 1977 western film directed by J. Lee Thompson and starring Charles Bronson, Kim Novak, Jack Warden, Slim Pickens and Will Sampson.[2] ...",https://en.wikipedia.org/wiki/Three_Silent_Men
1,0.09,A Dangerous Woman (1929 film),"A Dangerous Woman is a 1929 American Pre-Code film released by Paramount, based on the Margery Lawrence story, A Woman Who Needed Killing. It was directed by Gerald Grove and Rowland V. Lee from a scr ...",https://en.wikipedia.org/wiki/Ironweed_(film)
2,0.09,Paper Bullets,"Paper Bullets is a 1941 American film directed by Phil Rosen and starring Joan Woodbury. It was the first film produced by the King Brothers, launching their career.[1] The film was re-released by Eag ...",https://en.wikipedia.org/wiki/William_Shakespeare%27s_Romeo_%2B_Juliet
3,0.09,Fighting Caravans,"Fighting Caravans is a 1931 American pre-Code Western film directed by Otto Brower and David Burton and starring Gary Cooper, Lili Damita, and Ernest Torrence.[1] Based on the 1929 novel Fighting Cara ...",https://en.wikipedia.org/wiki/Prison_(1988_film)
4,0.09,The Showdown (1950 film),"The Showdown is a 1950 American Western film directed by Dorrell McGowan and Stuart E. McGowan and starring Wild Bill Elliott, Walter Brennan and Marie Windsor.[1][2][3] ...",https://en.wikipedia.org/wiki/Christopher_Strong
5,0.08,Joe (1970 film),"Joe is a 1970 American drama film distributed by Cannon Films[3] and starring Peter Boyle, Dennis Patrick and Susan Sarandon in her film debut. The film was directed by John G. Avildsen.[4][5] ...",https://en.wikipedia.org/wiki/Kokoda_(film)
6,0.08,Club Havana,Club Havana is a 1945 American film drama directed by Edgar G. Ulmer. It was produced and released by independent film company Producers Releasing Corporation. It has been compared to the 1933 film G ...,https://en.wikipedia.org/wiki/Dracula_2000
7,0.08,I Killed That Man,I Killed That Man is a 1941 American film directed by Phil Rosen that was a remake of his 1933 film The Devil's Mate. It starred Ricardo Cortez and was produced by the King Brothers. ...,https://en.wikipedia.org/wiki/Santa_with_Muscles
8,0.08,Chicago Calling,"Chicago Calling is a 1951 film noir directed by John Reinhardt and starring Dan Duryea and Mary Anderson. It centers on a man trying to find enough money to keep his phone connected, so he will be ab ...",https://en.wikipedia.org/wiki/Brand_Upon_the_Brain!
9,0.08,A Very Honorable Guy,"A Very Honorable Guy is a 1934 American pre-Code comedy film directed by Lloyd Bacon, written by Earl Baldwin, and starring Joe E. Brown, Alice White, Robert Barrat, Alan Dinehart, Irene Franklin and ...",https://en.wikipedia.org/wiki/Mississippi_Masala


## Task 3

In [125]:
#  Create new index and norm for movie names
name_index, name_norm = f.create_inverted_index_2([3], vocabulary, norm_file='name_norm.json', index_file='name_index.json')

In [169]:
def getAdditionalScore(df_search, additional_query, df_movies):
    df_movies.ID = df_movies.ID.astype(int)
    res = df_movies[df_movies.ID.isin(list(df_search.keys()))]
    additional_query_params = [x.strip() for x in additional_query.split(sep = ':')]
    if additional_query_params[0] == "year":
        res = res.assign(year = res["ID"].apply(addInfoYear))
        res = res.assign(diff_years = res["year"].apply(lambda x:addDiffYears( x, additional_query_params[1])))
        res = res.assign(points = res["diff_years"].apply(lambda x: addYearsPoints(x, res["diff_years"])))
    elif additional_query_params[0] == "language":
        res = res.assign(language = res["ID"].apply(addInfoLanguage))
        res = res.assign(points = res["language"].apply(lambda x: addLanguagePoints(x, additional_query_params[1])))
    elif additional_query_params[0] == "":
        res = res.assign(points = 0)
        print("No additional query")
    elif additional_query_params[0] == "":
        res = res.assign(points = 0)
        print("Additional query not valid!")
    additional_score = {}
    for index, row in res.iterrows():
        additional_score[row["ID"]] = row["points"]
    return additional_score

def addInfoYear(film_id):
    filename = f'tsv/{film_id}.tsv'
    film_data = pd.read_csv(filename, sep='\t', dtype=str, engine='python')
    release_date = film_data.iloc[0][" release date "]
    if "(" in release_date: 
        year =  release_date.split("(")[0].strip()[-4:]
        if len(year) == 4:
            return int(year)
    elif len(release_date.strip()) == 4:
        return int(release_date.strip())
    elif "," in release_date:
        year =  release_date.split(",")[1].strip()[-4:]
        if len(year) == 4:
            return int(year)
    elif " " in release_date.strip():
        info_date =  release_date.strip().split(" ")
        year = info_date[len(info_date) - 1].strip()
        if len(year) == 4:
            return int(year)
    return "NA"
    
def addInfoLanguage(film_id):
    filename = f'tsv/{film_id}.tsv'
    film_data = pd.read_csv(filename, sep='\t', dtype=str, engine='python')
    language = film_data.iloc[0][" language "]
    if language == "NA": 
        country = film_data.iloc[0][" country "]
        if country in ["United States", "England"]:
            return "English"
    else:
        return language
    return "NA"
         
def addDiffYears(film_year, film_query):
    diff = abs(film_year - int(film_query))
    return diff
    
def addLanguagePoints(language_code, language_query):
    if language_code.strip() == language_query.strip():
        return 1
    else:
        return 0
    
def addYearsPoints(current_diff, diff_years):
    max_diff = max(diff_years.values)
    if max_diff == 0:
        return 1
    else:
        return 1 - (current_diff / max_diff)    
    

In [170]:
doc_norm = f.load_doc_norm()

In [171]:
def search_3(query, top = 10):
    #  as_heap = False ==> returns scores as dictionary {document_id : score}
    #  strict_terms_filter = False ==> tells function to use not_strict_filter_documents
    name_results = f.search_with_tfidf(query, vocabulary, name_index, doc_norm, as_heap=False, strict_terms_filter=False)
    #  additional_score will be added to scores with the same document id
    additional_points = getAdditionalScore(name_results, input("Insert 'year:YYYY' or 'language: Language' to add additional info for the search \n"), df_movies)
    additional_score = {key: name_results.get(key, 0) + additional_points.get(key, 0) for key in set(name_results) | set(additional_points)}
    search_results = f.search_with_tfidf(query, vocabulary, inverted_index_2, doc_norm, additional_score=additional_points)
    return f.top_k(search_results, df_movies, top)
    

In [173]:
search_3('Sherlokc Holmes', 3)

Insert 'year:YYYY' or 'language: Language' to add additional info for the search 
year:2011


,Score,Title,Intro,URL
0,0.74,The Adventure of Sherlock Holmes' Smarter Brother,"The Adventure of Sherlock Holmes' Smarter Brother is a 1975 American musical comedy film with Gene Wilder, Marty Feldman, Madeline Kahn, Dom DeLuise, Roy Kinnear and Leo McKern.[3] The film was Wilder ...",https://en.wikipedia.org/wiki/The_Adventure_of_Sherlock_Holmes%27_Smarter_Brother
1,0.48,Sherlock Holmes in Washington,Sherlock Holmes in Washington (1943) is the fifth film in the Basil Rathbone/Nigel Bruce series of Sherlock Holmes movies. The plot is an original story not based on any of Sir Arthur Conan Doyle's Ho ...,https://en.wikipedia.org/wiki/Sherlock_Holmes_in_Washington
2,0.42,Sherlock Holmes Faces Death,"Sherlock Holmes Faces Death is the sixth film in the Basil Rathbone/Nigel Bruce series of Sherlock Holmes films.[1] Made in 1943, it is a loose adaptation of Sir Arthur Conan Doyle's Holmes story ""The ...",https://en.wikipedia.org/wiki/Sherlock_Holmes_Faces_Death


In [112]:
search_3('Sherlokc Holmes', 3)

,Score,Title,Intro,URL
0,3.11,The Adventures of Sherlock Holmes (film),The Adventures of Sherlock Holmes (released theatrically as Sherlock Holmes in the United Kingdom) is a 1939 mystery-adventure film released by Twentieth Century Fox.[2] It is a pastiche featuring the ...,https://en.wikipedia.org/wiki/The_Adventures_of_Sherlock_Holmes_(film)
1,2.88,Sherlock Holmes (1916 film),"Sherlock Holmes is a 1916 American silent film starring William Gillette as Arthur Conan Doyle's Sherlock Holmes. The film, which was directed by Arthur Berthelet, was produced by Essanay Studios in ...",https://en.wikipedia.org/wiki/Sherlock_Holmes_(1916_film)
2,2.81,Sherlock Holmes in Washington,Sherlock Holmes in Washington (1943) is the fifth film in the Basil Rathbone/Nigel Bruce series of Sherlock Holmes movies. The plot is an original story not based on any of Sir Arthur Conan Doyle's Ho ...,https://en.wikipedia.org/wiki/Sherlock_Holmes_in_Washington


In [111]:
search_3('Sherlokc Holmes', 3)

,Score,Title,Intro,URL
0,3.11,The Adventures of Sherlock Holmes (film),The Adventures of Sherlock Holmes (released theatrically as Sherlock Holmes in the United Kingdom) is a 1939 mystery-adventure film released by Twentieth Century Fox.[2] It is a pastiche featuring the ...,https://en.wikipedia.org/wiki/The_Adventures_of_Sherlock_Holmes_(film)
1,2.88,Sherlock Holmes (1916 film),"Sherlock Holmes is a 1916 American silent film starring William Gillette as Arthur Conan Doyle's Sherlock Holmes. The film, which was directed by Arthur Berthelet, was produced by Essanay Studios in ...",https://en.wikipedia.org/wiki/Sherlock_Holmes_(1916_film)
2,2.81,Sherlock Holmes in Washington,Sherlock Holmes in Washington (1943) is the fifth film in the Basil Rathbone/Nigel Bruce series of Sherlock Holmes movies. The plot is an original story not based on any of Sir Arthur Conan Doyle's Ho ...,https://en.wikipedia.org/wiki/Sherlock_Holmes_in_Washington
